# Qualidade de Dados

## Observação:
Os arquvivoos estão sendo salvos em formato Delta, devido a importancia da garantia da eficiência e confiabilidade dos dados. Dessa forma, temos consultas melhoradas e alterações seguras.

In [0]:
# Importa funções do PySpark para manipulação de colunas e cálculos
from pyspark.sql import functions as F

# ====================================================
# Função que gera relatório de qualidade de dados
# ====================================================
def data_quality_report(nome_tabela, fonte_dados, range_checks={}):
    """
    Gera um relatório de qualidade de dados para uma tabela Spark.

    Params:
        nome_tabela (str): Nome da tabela no catálogo (ex: "workspace.silver.pokemon")
        fonte_dados (str): Nome da fonte (ex: "pokemon", "spacex")
        range_checks (dict): Dicionário de checagens de range, ex:
                             {"stages": (1, 5), "date_utc": ("2000-01-01", "2030-01-01")}
    """

    # Lê a tabela Delta no catálogo a partir do nome passado
    df = spark.table(nome_tabela)

    # Conta o número total de linhas na tabela
    qtd_rows = df.count()

    # Inicializa lista para armazenar os percentuais de valores nulos por coluna
    null_ratios = []

    # Loop sobre todas as colunas da tabela
    for nome_coluna in df.columns:
        # Conta quantos valores nulos existem na coluna atual
        null_count = df.filter(F.col(nome_coluna).isNull()).count()

        # Calcula o percentual de nulos (se houver linhas)
        ratio = (null_count / qtd_rows) * 100 if qtd_rows > 0 else 0

        # Adiciona o percentual à lista de ratios
        null_ratios.append(ratio)

    # Calcula a média dos percentuais de nulos entre todas as colunas
    null_ratio_med = sum(null_ratios) / len(df.columns) if len(df.columns) > 0 else 0

    # Dicionário que armazenará os resultados dos range checks
    range_results = {}

    # Loop pelos campos que têm regras de range definidas
    for nome_coluna, (min_val, max_val) in range_checks.items():
        # Só aplica se a coluna existir na tabela
        if nome_coluna in df.columns:
            # Se os valores mínimos/máximos forem numéricos
            if isinstance(min_val, (int, float)):
                invalid_count = df.filter(
                    (F.col(nome_coluna) < min_val) | (F.col(nome_coluna) > max_val)
                ).count()
            # Se forem strings (datas ou formatos de texto)
            else:
                invalid_count = df.filter(
                    (F.col(nome_coluna) < F.lit(min_val)) | (F.col(nome_coluna) > F.lit(max_val))
                ).count()

            # Armazena no dicionário quantos valores estão fora do range
            range_results[nome_coluna] = invalid_count

    # Cria a linha do relatório com os resultados consolidados
    report = [(fonte_dados, qtd_rows, null_ratio_med, str(range_results))]

    # Converte a linha para um DataFrame Spark com colunas fixas
    report_df = spark.createDataFrame(
        report, ["FONTE_DADOS", "QTD_ROWS", "NULL_RATIO_MED", "RANGE_CHECKS"]
    )

    # Retorna o relatório em formato DataFrame
    return report_df

# ====================================================
# Executa relatório para a fonte Pokémon
# ====================================================
pokemon_report = data_quality_report(
    nome_tabela="workspace.silver.pokemon",   # Nome da tabela Silver com dados Pokémon
    fonte_dados="pokemon",                    # Nome da fonte
    range_checks={"url": ("https://pokeapi.co/api/v2/", "https://pokeapi.co/api/v2/zzzz")}  
    # Define um range "fake" só para verificar se o formato da URL está coerente
)

# ====================================================
# Executa relatório para a fonte SpaceX
# ====================================================
spacex_report = data_quality_report(
    nome_tabela="workspace.silver.spacex_launches",  # Nome da tabela Silver com lançamentos
    fonte_dados="spacex_launches",                   # Nome da fonte
    range_checks={"stages": (1, 5), "date_utc": ("2000-01-01", "2030-01-01")}  
    # Checa se estágios estão entre 1 e 5 e se as datas estão dentro do range esperado
)

# ====================================================
# Consolida relatórios em uma única tabela
# ====================================================
final_report = pokemon_report.union(spacex_report)  # Une os dois relatórios em um só DataFrame

# Salva os resultados no catálogo (camada de qualidade)
final_report.write.format("delta").mode("append").saveAsTable("workspace.quality.data_quality_report")

# Exibe os resultados finais no notebook
display(final_report)
